In [1]:
import json
import requests
import regex as re
from bs4 import BeautifulSoup

In [2]:
# Отримаємо дату
responce = requests.get('https://www.bbc.com/sport')
soup = BeautifulSoup(responce.text, features="lxml")

In [3]:
# Отримаємо посилання з основної сторінки
main_all_divs = soup.find_all("div")
main_divs = [div for div in main_all_divs if div.get("data-testid") == "promo" and div.get("type") == "article"]
main_links = ["https://www.bbc.com" + div.next.next.next.next.get("href") for div in main_divs if div.next.next.next.next.next.get("role") == "text"]
for i in main_links :
    print(i)
    print()

https://www.bbc.com/sport/football/articles/c80x1947rxzo

https://www.bbc.com/sport/athletics/articles/cqv5qde4w52o

https://www.bbc.com/sport/football/articles/c727z6nll05o

https://www.bbc.com/sport/cricket/articles/cyx5gyk33y6o

https://www.bbc.com/sport/football/articles/ckkg902ewqpo

https://www.bbc.com/sport/golf/articles/c51yq3lqd2vo

https://www.bbc.com/sport/football/articles/c1e5742g3nko

https://www.bbc.com/sport/football/articles/cnk4lyyyyd4o

https://www.bbc.com/sport/cycling/articles/cy68v8vg4pxo

https://www.bbc.com/sport/rugby-league/articles/c728v1rzpvlo

https://www.bbc.com/sport/football/articles/ckvgklgn4pyo

https://www.bbc.com/sport/sport-climbing/articles/cxe2p8dllk1o

https://www.bbc.com/sport/football/articles/c886pl8v9j8o

https://www.bbc.com/sport/golf/articles/c4ngw2x812go

https://www.bbc.com/sport/football/articles/ckvgz5v3weyo

https://www.bbc.com/sport/american-football/articles/cv2gk12w3y2o

https://www.bbc.com/sport/football/articles/c10lq33j7myo

http

In [4]:
# Беремо перші 5
links = main_links[:5]
responces = [requests.get(link).text for link in links]
soups = [BeautifulSoup(res, features="lxml") for res in responces]

In [5]:
# Беремо зв'язані теми
yield_titles = []
yield_links = []
yield_local_titles = []
yield_local_links = []

for i in range(0, len(links)):
    h2_raw = soups[i].find_all("h2")
    h2 = None
    for h2s in h2_raw :
        if h2s.text == "Related Topics" :
            h2 = h2s # Існує лише один підходящий h2
    if h2 == None :
        print("No related topics! ID :", main_links[i])
        continue
    related_topics_raw = h2.next.next.next

    related_topics_links = re.findall("href=\"([\w \/=+-]{1,})", str(related_topics_raw))
    related_topics_links = ["https://www.bbc.com" + link for link in related_topics_links]

    related_topics_titles = re.findall("([\w \"\'\/=+-]{1,})<\/a>", str(related_topics_raw))
    
    yield_local_titles = []
    yield_local_links = []
    
    for j in range(0, len(related_topics_links)) :
        yield_local_titles.append(related_topics_titles[j])
        yield_local_links.append(related_topics_links[j])
    
    yield_titles.append(yield_local_titles)
    yield_links.append(yield_local_links)

In [8]:
# Зберігаємо
filename = "JSON.json"
data = [
    {"links" : yield_links[i], "titles": yield_titles[i]}
    for i in range(0, len(yield_titles))
]
with open(filename, "w", encoding='utf-8') as f:
    json.dump(data, f, indent=2)